In [1]:
"""
DS4420: Data Mining & Machine Learning 2
Final Project
Calvin Li & Rishita Shroff

Data Collection & Pre-processing
"""

'\nDS4420: Data Mining & Machine Learning 2\nFinal Project\nCalvin Li & Rishita Shroff\n\nData Collection & Pre-processing\n'

In [2]:
# importing required libraries
import numpy as np
import pandas as pd

import datetime as dt
import yfinance as yf
import pandas_datareader.data as web
import cpi
#cpi.update()
import wbdata

C:\Users\rishi\AppData\Local\Temp\ipykernel_27436\2906279748.py:8: StaleDataWarning: CPI data is out of date. To accurately inflate to today's dollars, you must run `cpi.update()`.
  import cpi


In [4]:
# importing sp500 list of stocks
sp500_stocks = pd.read_csv("sp500_stocks.csv", encoding="latin-1")
#print(sp500_stocks)

# customize seed for different stocks
#np.random.seed(42)

# list of all potential stocks 
#ticker_list = list(sp500_stocks["Ticker"])

# using ticker list as stock list coz want to run idea by calvin
stock_list = list(sp500_stocks["Ticker"])

# selecting 15 random stocks
#stock_list = np.random.choice(ticker_list,15,replace=False)
#stock_list = list(np.sort(stock_list))
#print(f'These are the fifteen stocks assigned to you: {" ".join(stock_list)}')

# defining start and end date
start = dt.datetime(2000, 1, 1)
end = dt.datetime(2024, 12, 31)

In [5]:
# getting returns of the stocks
# calculates returns using adjusted close price
# ensures that we have returns of only those dates where all companies have available information
#returns = yf.download(stock_list, start-pd.offsets.BDay(1), end+pd.offsets.BDay(1), auto_adjust=False)['Adj Close'].pct_change().dropna()

# getting prices of the stocks
prices = yf.download(stock_list, start-pd.offsets.BDay(1), end+pd.offsets.BDay(1), auto_adjust=False)['Adj Close']

[*********************100%***********************]  503 of 503 completed


In [6]:
# defining empty list
filtered_stocks = []

for i in prices.columns:
    # getting stocks who had some prices in 2005 jan
    if prices.loc[pd.Timestamp("2005-01-03"), i] >= 0:

        # adding them to filtered list
        filtered_stocks.append(i)

In [7]:
# getting df
filtered_df = prices[filtered_stocks]
#print(filtered_df)

# dropping null values
filtered_df = filtered_df.dropna()

In [8]:
# getting daily returns
returns = filtered_df.pct_change().dropna()
#returns

In [9]:
# S&P 500
sp500 = yf.Ticker("^GSPC")
sp500_data = sp500.history(period="21y")
sp500_data['SP500'] = sp500_data['Close'].pct_change()
sp500_data = sp500_data.drop(columns=["Open", "High", "Low", "Close", "Volume", "Dividends", "Stock Splits"])

sp500_data.index = sp500_data.index.strftime('%m-%d-%Y')
sp500_data.index = pd.to_datetime(sp500_data.index)
# sp500_mon = sp500_data.resample(rule = 'ME').apply(lambda x: x.add(1).prod().sub(1))

#sp500_data

In [10]:
# Find finding monthly returns
returns = pd.merge(returns, sp500_data, left_index = True, right_index = True )

returns_mon = returns.resample(rule = 'ME').apply(lambda x: x.add(1).prod().sub(1))

In [11]:
returns_mon

,A,AAPL,ABT,ACGL,ACN,ADBE,ADI,ADM,ADP,ADSK,...,WST,WTW,WY,WYNN,XEL,XOM,YUM,ZBH,ZBRA,SP500
Date,,,,,,,,,,,,,,,,,,,,,
2004-12-31,-0.003309,-0.013178,0.051860,-0.005141,-0.012797,0.001646,-0.028165,0.029059,0.002713,0.090789,...,0.045530,0.002619,-0.000595,0.004352,8.103825e-03,0.014848,0.020329,-0.016933,0.019381,0.005142
2005-01-31,-0.082572,0.194099,-0.029582,-0.050904,-0.035185,-0.093083,-0.027899,0.084716,-0.019617,-0.226087,...,0.047276,-0.060481,-0.071705,-0.020323,-5.491063e-04,0.006633,-0.015475,-0.015851,-0.095060,-0.025290
2005-02-28,0.085481,0.166711,0.021546,0.129322,-0.019194,0.085237,0.024811,-0.000679,-0.011959,0.011917,...,-0.015326,0.022492,0.079472,0.091672,-2.583866e-02,0.232947,0.052427,0.089411,-0.020813,0.018903
2005-03-31,-0.075000,-0.071111,0.013698,-0.034716,-0.054794,0.087987,-0.015796,0.019917,0.050084,0.001862,...,-0.070039,-0.062193,0.023457,-0.053514,-1.840762e-02,-0.058601,0.062115,-0.094179,-0.047724,-0.019118
2005-04-30,-0.065315,-0.134629,0.060572,-0.001249,-0.101449,-0.114634,-0.056170,-0.268104,-0.033593,0.069556,...,0.107255,-0.092758,0.001605,-0.218482,-3.330669e-16,-0.043121,-0.091839,0.046395,0.005685,-0.020109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-31,0.010750,0.032353,0.069190,0.180727,0.034269,0.041258,0.014954,-0.008044,0.050606,0.043956,...,0.024366,0.034824,-0.033649,-0.068670,5.061770e-02,0.002526,0.020802,0.036911,-0.016544,0.022835
2024-09-30,0.038903,0.017467,0.006533,-0.010699,0.033718,-0.098588,-0.016031,-0.020495,0.008041,0.066099,...,-0.042949,0.011322,0.110528,0.247138,7.569739e-02,-0.006105,0.035503,-0.062966,0.072210,0.020197
2024-10-31,-0.120976,-0.030429,-0.000927,-0.119056,-0.020529,-0.076674,-0.030673,-0.075829,0.045206,0.030202,...,0.025886,0.026007,-0.079740,0.001460,2.312391e-02,-0.003754,-0.061198,-0.009541,0.031459,-0.009897


In [12]:
# Standard deviation for 2018, May SP500 
sp500_stdev = returns.loc[(returns.index.year == 2018) & (returns.index.month == 5)]['SP500'].std()
print(f"The standard deviation for SP500 in May of 2018 is {sp500_stdev}")

# Calculating STDEV for each month
monthly_std = returns.groupby(by=[returns.index.month, returns.index.year]).std()
monthly_std.index = pd.to_datetime(monthly_std.index.map(lambda x: f"{x[1]}-{x[0]}"), format="%Y-%m")
monthly_std.sort_index(inplace=True)
monthly_std.index = returns_mon.index

# Double check 
monthly_std

The standard deviation for SP500 in May of 2018 is 0.006550294321030806


,A,AAPL,ABT,ACGL,ACN,ADBE,ADI,ADM,ADP,ADSK,...,WST,WTW,WY,WYNN,XEL,XOM,YUM,ZBH,ZBRA,SP500
Date,,,,,,,,,,,,,,,,,,,,,
2004-12-31,0.011183,0.017300,0.015261,0.010422,0.009260,0.013331,0.011534,0.013200,0.007678,0.023779,...,0.031860,0.010922,0.005319,0.021756,0.008197,0.012592,0.005305,0.012896,0.011176,0.004724
2005-01-31,0.015337,0.028358,0.010376,0.007567,0.016653,0.013558,0.014966,0.020805,0.013980,0.037855,...,0.009331,0.011721,0.010742,0.024634,0.007796,0.009590,0.009465,0.012896,0.019924,0.006447
2005-02-28,0.018691,0.022854,0.009947,0.012481,0.013030,0.021544,0.020864,0.021207,0.009512,0.026762,...,0.015225,0.011469,0.013033,0.019286,0.012277,0.014524,0.011437,0.027411,0.022322,0.006730
2005-03-31,0.013886,0.022648,0.010223,0.008496,0.011023,0.018118,0.015624,0.013200,0.004762,0.013132,...,0.016012,0.010998,0.011658,0.020187,0.010194,0.014474,0.013788,0.017878,0.015793,0.006339
2005-04-30,0.015846,0.035998,0.014212,0.007616,0.014505,0.031022,0.014443,0.047626,0.012181,0.023370,...,0.040541,0.013907,0.021138,0.027074,0.008504,0.020634,0.013108,0.014508,0.015706,0.009503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-31,0.013861,0.014390,0.011846,0.012121,0.011381,0.016783,0.029006,0.010993,0.008371,0.018378,...,0.018602,0.010168,0.012885,0.021677,0.012369,0.013174,0.009026,0.015038,0.019970,0.012262
2024-09-30,0.016580,0.014810,0.011136,0.010678,0.018660,0.023263,0.024488,0.016036,0.009211,0.011440,...,0.018413,0.007523,0.013204,0.027843,0.007073,0.016501,0.013090,0.021166,0.018367,0.008719
2024-10-31,0.011671,0.013200,0.009935,0.022443,0.013401,0.015289,0.017170,0.012342,0.007570,0.012356,...,0.038040,0.012459,0.014881,0.019485,0.015597,0.012030,0.007329,0.017107,0.015557,0.006936


In [13]:
# Finding other features we care about 
# Finding risk free-rate
rf = web.DataReader('F-F_Research_Data_Factors','famafrench', start, end)[0][['RF']].div(100)
rf.index = rf.index.to_timestamp(how='end').normalize()

# CPI data from FRED
cpi = web.DataReader('CPIAUCSL', 'fred', start, end)

# Calculate monthly inflation rates as percentage change
inflation_rate = cpi.pct_change().dropna()
inflation_rate.index = inflation_rate.index.to_period('M').to_timestamp(how='end').normalize()
inflation_rate.columns = ['Inflation Rate']

# Rolling Average
rolling_avg = returns.rolling(window=30).mean()
rolling_avg.columns = [name+"_ROLL_AVG" for name in rolling_avg.columns]
rolling_monthly = rolling_avg.resample('ME').last()

C:\Users\rishi\AppData\Local\Temp\ipykernel_27436\2274658707.py:3: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  rf = web.DataReader('F-F_Research_Data_Factors','famafrench', start, end)[0][['RF']].div(100)
C:\Users\rishi\AppData\Local\Temp\ipykernel_27436\2274658707.py:3: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  rf = web.DataReader('F-F_Research_Data_Factors','famafrench', start, end)[0][['RF']].div(100)


In [14]:
# Creating the final df 
result = pd.concat([returns_mon, monthly_std], axis=1, keys=['RETURN', 'STDEV'])
result.columns = result.columns.get_level_values(1) + '_' + result.columns.get_level_values(0).astype(str)

result = pd.merge(result, rf, how='left', left_index=True, right_index=True)
result = pd.merge(result, inflation_rate, how = 'left', left_index=True, right_index=True) 
result = pd.merge(result, rolling_monthly, how = 'left', left_index=True, right_index=True)

# Removing first two rows because rolling average has errors
result = result.iloc[2:]

result

,A_RETURN,AAPL_RETURN,ABT_RETURN,ACGL_RETURN,ACN_RETURN,ADBE_RETURN,ADI_RETURN,ADM_RETURN,ADP_RETURN,ADSK_RETURN,...,WST_ROLL_AVG,WTW_ROLL_AVG,WY_ROLL_AVG,WYNN_ROLL_AVG,XEL_ROLL_AVG,XOM_ROLL_AVG,YUM_ROLL_AVG,ZBH_ROLL_AVG,ZBRA_ROLL_AVG,SP500_ROLL_AVG
Date,,,,,,,,,,,,,,,,,,,,,
2005-02-28,0.085481,0.166711,0.021546,0.129322,-0.019194,0.085237,0.024811,-0.000679,-0.011959,0.011917,...,0.000559,0.000067,0.000489,0.003326,-0.000229,0.007779,0.002467,0.003826,-0.000568,0.000754
2005-03-31,-0.075000,-0.071111,0.013698,-0.034716,-0.054794,0.087987,-0.015796,0.019917,0.050084,0.001862,...,-0.003771,-0.000624,0.003027,-0.002970,-0.001402,0.001677,0.002581,-0.002082,-0.001548,-0.000801
2005-04-30,-0.065315,-0.134629,0.060572,-0.001249,-0.101449,-0.114634,-0.056170,-0.268104,-0.033593,0.069556,...,0.003700,-0.003257,0.000897,-0.010352,0.000075,-0.002336,-0.002288,0.000646,0.000080,-0.000911
2005-05-31,0.157108,0.102607,-0.018715,0.116529,0.072811,0.113839,0.090029,0.108213,0.008287,0.243481,...,0.006289,-0.000905,-0.000478,-0.007291,0.002739,-0.000279,0.001147,-0.000026,-0.001984,0.001331
2005-06-30,-0.041233,-0.074195,0.015962,0.008959,-0.026203,-0.136171,0.006203,0.077078,-0.038370,-0.132390,...,-0.000370,-0.000877,-0.000675,0.000642,0.002916,0.002375,0.000457,-0.000513,0.001724,0.000173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-31,0.010750,0.032353,0.069190,0.180727,0.034269,0.041258,0.014954,-0.008044,0.050606,0.043956,...,0.000131,0.003565,0.000710,-0.002428,0.003086,0.000884,0.002249,0.001546,0.002595,0.000928
2024-09-30,0.038903,0.017467,0.006533,-0.010699,0.033718,-0.098588,-0.016031,-0.020495,0.008041,0.066099,...,0.000150,0.001766,0.004450,0.008028,0.003153,-0.000145,0.000781,-0.000587,0.002576,0.001259
2024-10-31,-0.120976,-0.030429,-0.000927,-0.119056,-0.020529,-0.076674,-0.030673,-0.075829,0.045206,0.030202,...,0.000184,0.001671,-0.002397,0.006385,0.001493,0.000308,0.000080,0.000368,0.001723,-0.000029
